#### This kernel used dataset from the Zillow Prize: Zillow’s Home Value Prediction and copied from the 'Simple XGBoost Starter' written by anokas.
#### Introduction to 'Simple XGBoost Starter' : [URL](https://www.kaggle.com/anokas/simple-xgboost-starter-0-0655)
#### Thanks for sharing kernel, anokas

In [ ]:
# import library
import numpy as np
import pandas as pd
import xgboost as xgb
import gc
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('../input/zillow-prize-1/train_2016_v2.csv')
prop = pd.read_csv('../input/zillow-prize-1/properties_2016.csv')
sample = pd.read_csv('../input/zillow-prize-1/sample_submission.csv')

In [ ]:
for c, dtype in zip(prop.columns, prop.dtypes):
    if dtype == np.float64:
        prop[c] = prop[c].astype(np.float32)

print('creating training set ...')

In [ ]:
df_train = train.merge(prop, how='left', on='parcelid')

x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode'], axis=1)
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)

In [ ]:
train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

del df_train; gc.collect()

In [ ]:
split = 80000
x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]

print('Building DMatrix...')

In [ ]:
# DMatrix : 넘파이 입력 파라미터를 받아서 만들어지는 XGBoost만의 전용 데이터 세트

# 주요 입력 파라미터는 data(피처 데이터 세트)와 label
# (분류: 레이블 데이터 세트/회귀: 숫자형인 종속값 데이터 세트)

d_train = xgb.DMatrix(x_train, label = y_train)
d_valid = xgb.DMatrix(x_valid, label = y_valid)

del x_train, x_valid; gc.collect()

In [ ]:
params = {}
params['eta'] = 0.02
params['objective'] = 'reg:linear'
params['eval_metric'] = 'mae'
params['max_depth'] = 4
params['silent'] = 1

In [ ]:
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
clf = xgb.train(params, d_train, 10000, watchlist,
                early_stopping_rounds=100, verbose_eval=10)

del d_train, d_valid

print('Bulidng test set ...')

In [ ]:
sample['parcelid'] = sample['ParcelId']
df_test = sample.merge(prop, on='parcelid', how = 'left')

del prop; gc.collect()

In [ ]:
x_test = df_test[train_columns]
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)
    
del df_test, sample; gc.collect()

In [ ]:
d_test = xgb.DMatrix(x_test)

del x_test; gc.collect()

print('Predicting on test ...')

In [ ]:
p_test = clf.predict(d_test)

del d_test; gc.collect()

In [ ]:
sub = pd.read_csv('../input/zillow-prize-1/sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = p_test

print('Writting csv ...')

sub.to_csv('xgb_starter.csv', index=False, float_format='%.4f')